In [37]:

import os
import glob

import textgrid
import numpy as np

# Load the TextGrid file
def load_textgrid(file_path):
    try:
        tg = textgrid.TextGrid.fromFile(file_path)
        # Iterate through tiers
        for tier in tg.tiers:
            for interval in tier:
                if interval.mark != '':
                    res = (interval.minTime, interval.maxTime, interval.mark)
            

            return res, int(tier[-1].maxTime * 100)
    except Exception as e:
        print(f"An error occurred: {e} at error file: {file_path}")
        return None, None


folder = 'wavs_single_channel/EarlyLate-features/vad2/'
files = glob.glob(os.path.join(folder, '*.npy'))
diff = []
cnt_empty = 0
for fn in files:
    if 'prac' in fn:
        continue
    vadres = np.load(fn)
    if len(vadres) <= 15:
        raise ValueError(f"File too short: {fn  }")
    # find first and last index of True
    start = np.argmax(vadres[15:]) / 100.0
    start += 0.15
    end = (len(vadres) - np.argmax(np.flip(vadres))) / 100.0
    tgfn = os.path.basename(fn)
    tgfn = tgfn.split('.')[0] + '_' + tgfn.split('.')[1] + '.TextGrid'
    if tgfn is not None:
        tgfn = os.path.join('wavs_single_channel/EarlyLate-TG/', os.path.basename(tgfn))
    tg, length = load_textgrid(tgfn)
    if tg is None:
        continue
    
    if length != len(vadres):
        print(f"Length mismatch: {length} vs {len(vadres)}")
    else:
        if end - start < 0.08:
            cnt_empty += 1
            
        diff.append([abs(start - tg[0]), abs(end - tg[1])])

# calculate the max difference and average difference

print(f"cnt_empty: {cnt_empty}")
diff = np.array(diff)
print(np.max(diff, axis=0))
# get first and second column max

print(np.max(diff[:, 1]))
print(np.max(diff[:, 0]))
# count how many are greater than 2.0
print(np.sum(diff[:, 1] > 2.0))
print(np.sum(diff[:, 0] > 2.0))
print(np.sum(diff[:, 1] > 5.0))
print(np.sum(diff[:, 0] > 5.0))


print(np.mean(diff, axis=0))

# remove the outliers > 2.0
diff = diff[diff[:, 1] < 2.0]
diff = diff[diff[:, 0] < 2.0]

print(np.mean(diff, axis=0))
    


An error occurred: local variable 'res' referenced before assignment at error file: wavs_single_channel/EarlyLate-TG/subj-2209_48_L_praten_veren_wav_1.TextGrid


KeyboardInterrupt: 